In [1]:
import os
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
from osgeo import gdal

Generate samples from images

In [2]:
gdal.AllRegister()

def getData(raster):
    raster = gdal.Open(raster)
    cols = raster.RasterXSize
    rows = raster.RasterYSize
    bands = raster.RasterCount
    datas=[]
    for i in range(bands):
        band = raster.GetRasterBand(i+1)
        NoData = band.GetNoDataValue()
        data = band.ReadAsArray(0, 0, cols, rows)
        data[data==NoData]=0
        datas.append(data)
    datas=np.array(datas)
    return datas

image=getData('images\Image.tif')[:3,:,:].transpose(1,2,0)
dsm=getData('images\dsm.tif').transpose(1,2,0)
mask=getData('images\mask.tif').transpose(1,2,0)
mask[mask==2]=1 # Binary classification

ps = 128  # patch size

# sample
for h in range(0, image.shape[0] - ps, int(ps / 2)):
    for w in range(0, image.shape[1] - ps, int(ps / 2)):
        if dsm[h, w, :] != 0:
            img_patch = image[h : h + ps, w : w + ps, :]
            mask_patch = mask[h : h + ps, w : w + ps, :]
            dsm_patch = dsm[h : h + ps, w : w + ps, :]
            img_patch = np.concatenate((img_patch, dsm_patch), -1)#使用DSM
            patch_dict = {}
            patch_dict["image"] = img_patch
            patch_dict["mask"] = mask_patch
            dict_name = "%d_%d.mat" % ( h, w)
            scipy.io.savemat('dataset/samples/'+dict_name, patch_dict)
